# PCA - Principal Component Analysis

In [1]:
# Includes:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# utility functions for the project
def vcol(v):
    return np.array(v).reshape(v.size, 1)

def vrow(v):
    return np.array(v).reshape(1, v.size)



SyntaxError: incomplete input (3679904627.py, line 2)